In [1]:
import os
import warnings
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestClassifier, RandomForestRegressor,
    AdaBoostClassifier, AdaBoostRegressor,
    GradientBoostingClassifier, GradientBoostingRegressor
)
from xgboost import XGBClassifier, XGBRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR

from sklearn.preprocessing import (
    StandardScaler, MinMaxScaler,
    LabelEncoder, OneHotEncoder,
    QuantileTransformer, PowerTransformer
)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    classification_report, accuracy_score, f1_score, precision_score
)

warnings.filterwarnings('ignore')

print("Core Libraries: pandas, numpy, matplotlib.pyplot, seaborn, warnings")
print("Train/Test Split: train_test_split, GridSearchCV, RandomizedSearchCV")
print("Models: GaussianNB, BernoulliNB, MultinomialNB, DecisionTreeClassifier, DecisionTreeRegressor, "
      "RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier, AdaBoostRegressor, "
      "GradientBoostingClassifier, GradientBoostingRegressor, XGBClassifier, XGBRegressor, "
      "LogisticRegression, LinearRegression, KNeighborsClassifier, KNeighborsRegressor, SVC, SVR")
print("Preprocessing: StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder, "
      "QuantileTransformer, PowerTransformer, ColumnTransformer, Pipeline")
print("Metrics: mean_squared_error, mean_absolute_error, r2_score, "
      "classification_report, accuracy_score, f1_score, precision_score")
print("Other: pickle")


Core Libraries: pandas, numpy, matplotlib.pyplot, seaborn, warnings
Train/Test Split: train_test_split, GridSearchCV, RandomizedSearchCV
Models: GaussianNB, BernoulliNB, MultinomialNB, DecisionTreeClassifier, DecisionTreeRegressor, RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier, AdaBoostRegressor, GradientBoostingClassifier, GradientBoostingRegressor, XGBClassifier, XGBRegressor, LogisticRegression, LinearRegression, KNeighborsClassifier, KNeighborsRegressor, SVC, SVR
Preprocessing: StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder, QuantileTransformer, PowerTransformer, ColumnTransformer, Pipeline
Metrics: mean_squared_error, mean_absolute_error, r2_score, classification_report, accuracy_score, f1_score, precision_score
Other: pickle


In [2]:
def train_random_forest(data, target):
    # Dictionary to store LabelEncoders for each categorical column
    label_encoders = {}

    # split the data into X and y
    X = data.drop(target, axis=1)
    y = data[target]

    # Create a new LabelEncoder for each categorical column
    for col in X.select_dtypes(include=['object', 'category']).columns:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col])
        label_encoders[col] = le

    # split the data into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    # Scaling Data
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Define the Random Forest model
    rf_model = RandomForestClassifier(random_state=0,class_weight='balanced')

    # Define hyperparameters for tuning
    param_grid = {
        'n_estimators': [50, 100, 150],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    # Perform GridSearchCV for hyperparameter tuning
    grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # Get the best model and parameters
    best_rf_model = grid_search.best_estimator_
    best_params = grid_search.best_params_

    # Print the best hyperparameters
    print('Best Hyperparameters:')
    print(best_params)

    # Train the model on the full training set
    best_rf_model.fit(X_train, y_train)

    # Evaluate the model on the test set
    y_pred = best_rf_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f'Accuracy on Test Set: {accuracy:.2f}')

    # Inverse transform at the End
    # Loop through each column to decode the data
    for col, le in label_encoders.items():
        # Use the inverse_transform method to decode the column in both training and test sets
        X[col] = le.inverse_transform(X[col])

    return best_rf_model, best_params, accuracy
